In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import ModelCheckpoint # type: ignore
from keras import metrics

In [ ]:
df = pd.read_csv('Data_In.csv')
df.rename(columns = {'company_name' : 'Company'}, inplace = True)

# Print Rows with Null Values
# df[df.isnull().any(axis=1)]

In [ ]:
factor = 'Company'
search_term = 'AMAZON'

# Filter rows where 'Type' is equal to 'Health'
filtered_df = df[df[factor] == search_term]
filtered_df.shape


In [ ]:
# Reshape data into a 1D array
# Scale data to the min and max of the rows i.e 0 -1 range
# Flatten the data once again to a 1D array
# ====================================================================

data = filtered_df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [245]:
def create_sequences(data, seq_length):
    sequences, labels = [], []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length, 0]
        label = data[i:i + seq_length, 0]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [246]:
seq_length = 1
n_samples = len(data_scaled)
sequences, labels = create_sequences(data_scaled, seq_length)
split = int(0.8 * len(sequences))
X_test, y_test = sequences[split:], labels[split:]
X_train, y_train = sequences[:split], labels[:split]

In [247]:
model_lstm = Sequential()
model_lstm.add(LSTM(100, activation='relu', input_shape=(seq_length, 1), return_sequences=True))
model_lstm.add(LSTM(50, activation='relu'))
model_lstm.add(Dense(320))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

/home/allan/.virtualenvs/DSC/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [248]:
model_lstm.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - loss: 0.0048 - val_loss: 3.1570e-04
Epoch 2/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 1.2811e-06 - val_loss: 2.1318e-04
Epoch 3/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 1.0435e-06 - val_loss: 1.6111e-04
Epoch 4/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 4.7093e-07 - val_loss: 8.5289e-05
Epoch 5/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 2.6891e-07 - val_loss: 7.1788e-05
Epoch 6/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.0917e-07 - val_loss: 4.6343e-05
Epoch 7/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 4.9643e-08 - val_loss: 4.5753e-05
Epoch 8/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 5.4329e-08 - val_loss: 3.9362e-05
Epoch 9/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 3.3577e-08 - val_loss: 3.5036e-05
Epoch 10/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 3.2360e-08 - val_loss: 2.9898e-05
Epoch 11/40
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/ste

In [249]:
model_lstm.save('model_lstm_7.keras')

In [250]:
predictor = load_model('model_lstm_7.keras')
amazn = predictor.predict(X_test)

42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [251]:
amazn = scaler.inverse_transform(amazn)
amazn_y = scaler.inverse_transform(y_test.reshape(-1, 1))

In [253]:
dates_test = filtered_df['Date'].iloc[split + seq_length:]
# df_plot_lstm = pd.DataFrame({'Fechas': dates_test, 'Real': y_test_actual_LSTM.flatten(), 'Predict': predictions__LSTM.flatten()})
df_plot_lstm = pd.DataFrame({'Date': dates_test[:len(amazn)], 'Real': amazn_y.flatten(), 'Predict': amazn.flatten()})

df_plot_lstm.tail(50)
df_plot_lstm.to_csv('AMAZON.csv')